In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Завантаження та попередня обробка даних
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Нормалізація даних

# Побудова та конфігурація моделі VGG16 без верхніх шарів
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Заморожуємо деякі згорткові шари для виділення ознак
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Додаємо верхні шари для класифікації Fashion MNIST
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(10, activation='softmax')(x)

# Складаємо фінальну модель
model = Model(inputs=base_model.input, outputs=output)

# Компіляція моделі та конфігурація оптимізатора
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Збільшуємо розмір зображень для VGG16
x_train_upscaled = tf.image.grayscale_to_rgb(tf.convert_to_tensor(x_train))
x_test_upscaled = tf.image.grayscale_to_rgb(tf.convert_to_tensor(x_test))

# Визначаємо параметри для генерації додаткових даних
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)

# Навчання моделі з використанням прийому донавчання
batch_size = 32
epochs = 50

history = model.fit(
    datagen.flow(x_train_upscaled, y_train, batch_size=batch_size),
    validation_data=(x_test_upscaled, y_test),
    epochs=epochs,
    verbose=2
)

# Візуалізація точності та функції втрат під час навчання
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Оцінка точності на тестових даних
test_loss, test_acc = model.evaluate(x_test_upscaled, y_test, verbose=2)
print("\nТочність на тестових даних:", test_acc)
